In [1]:
# LIBRARIES
# *********
import requests # request for the web pagegg
from fake_useragent import UserAgent # to fake meta data
from requests.packages.urllib3.exceptions import InsecureRequestWarning # handle while requesting for the web page
from selenium import webdriver # create a simulation of the pseudo-user
import time # set sleep on web page on every click, handling nginx error
import datetime  # record start time
from bs4 import BeautifulSoup # to get web page soure
import pandas as pd # create df
import numpy as np # manipulate with shape of the df
import pyodbc
import csv # get rid off quotes
import json
lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
d = webdriver.Chrome(chrome_options=options, executable_path=r"\\aafssrv\SHARED-BI\driver\chromedriver.exe")

<ipython-input-2-d89da3cdb56e>:4: DeprecationWarning: use options instead of chrome_options
  d = webdriver.Chrome(chrome_options=options, executable_path=r"\\aafssrv\SHARED-BI\driver\chromedriver.exe")


In [3]:
# L I N K S
# baseURL of Qoldau
baseUrl = "https://subsidies.qoldau.kz/ru/subsidies/"

# applications
appFerti = "fertilizer/pub-apps"
appPest = "herbicides/pub-apps"
appSeed = "seed/pub-apps"
y2019 = "?Year=2019"

listOfLinks = [appPest, appFerti, appPest+y2019, appFerti+y2019] #appSeed
listOfNames =  [ "Пестициды2020", "Удобрения2020", "Пестициды2019", "Удобрения2019"] #"Семена2019_2020"

linkN = 0
for link in listOfLinks:
    print(linkN, " = ", listOfNames[linkN])
    print("Started: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
    d.get(baseUrl+link)#, headers={'User-Agent': UserAgent().chrome})
    time.sleep(2)
    html = d.page_source
    nPages = int(d.find_elements_by_xpath('//li[@data-page-index]/a')[-2].text)
    print("This page's Navgation steps = ", nPages)
    tempLastN = []
    tempRowData = []
    cols = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table").find_elements_by_tag_name('th')
    colNames = []
    tempColNames = []
    for col in cols:
        r = col.get_attribute("innerText").replace('\n', ',')
        tempColNames.append(r)
    colN = len(tempColNames)
    print("Number of cols = ", colN, "Columns are ", tempColNames)
    for LastN in d.find_elements_by_xpath("//div[@class='pull-right']/div/span"):
        tempLastN.append(LastN.text.split(' '))
    lastN = int(tempLastN[-1][-1])
    someone = 0
    someonee = 0
    for click in range(nPages):
        rows = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table/tbody").find_elements_by_tag_name('td')
        if(rows[0].text != "Нет записей"):
            for row in rows:
                tempRowData.append(row.text)
            d.find_elements_by_xpath('//li[@data-page-index]/a')[-1].click()
            someone+=1
            time.sleep(3)
            if someone == 700:
                f = pd.DataFrame(lol(tempRowData, colN), columns=tempColNames)
                f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\qld(new)\/' + listOfNames[linkN] + '-' + str(someonee) + '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
                tempRowData = []
                print("Finished: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
                someone = 0
                someonee+=1
    f = pd.DataFrame(lol(tempRowData, colN), columns=tempColNames)
    if someonee == 0:
        f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\qld(new)\/' + listOfNames[linkN] + '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
    else:
        f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\qld(new)\/' + listOfNames[linkN] + '-' + str(someonee) + '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
    linkN += 1
    tempRowData = []
    print("Finished: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))

0  =  Пестициды2020
Started:  2020-12-22, 19:06:18
This page's Navgation steps =  1688
Number of cols =  16 Columns are  ['№', 'Область', 'Дата подачи заявки', 'Наименование заявителя', 'Производитель пестицидов', 'Наименование препарата', 'Объем пестицида в базовой ед.измерения', 'Базовая единица измерения', 'Площадь использования, га', 'Статус', 'Тип заявки', 'Дата принятия заявки', 'Дата отклонения', 'Сумма субсидий (расчетная), тг.', 'Причина отклонения', 'Примечание к отрицательному решению']
Finished:  2020-12-22, 21:09:22
Finished:  2020-12-22, 23:11:29
Finished:  2020-12-23, 00:01:42
1  =  Удобрения2020
Started:  2020-12-23, 00:01:42
This page's Navgation steps =  953
Number of cols =  16 Columns are  ['№', 'Область', 'Наименование заявителя', 'Дата подачи заявки', 'Статус', 'Площадь использования, га', 'Производитель удобрений', 'Наименование удобрения', 'Единица измерения', 'Объем удобрения', 'Сумма субсидий (расчетная), тг.', 'Тип заявки', 'Дата принятия заявки', 'Дата отриц